# 🎯 RIQ MACHINE - NBA Prediction & Analysis

## Features:
- ✅ Load trained neural hybrid models (TabNet + LightGBM)
- ✅ Fetch live NBA games and player stats
- ✅ Generate predictions with full feature engineering
- ✅ Compare to betting lines (find value)
- ✅ Backtest on historical data
- ✅ Track performance and ROI

## Prerequisites:
1. Trained models from `NBA_COLAB_SIMPLE.ipynb`
2. Three separate files to upload:
   - `nba_models_trained.zip` (trained models)
   - `priors_data.zip` (Basketball Reference priors)
   - `PlayerStatistics.csv` (historical data for backtesting)

## Quick Start:
1. Run Setup cell
2. Upload 3 files (separate prompts)
3. Choose: Backtest OR Live Predictions

---

## 📦 Setup & Installation

In [ ]:
# ============================================================
# SETUP - Install packages and download code
# ============================================================

print("📦 Installing packages...")
!pip install -q nba-api pytorch-tabnet lightgbm scikit-learn pandas numpy requests

print("\n📥 Downloading training code (for feature engineering)...")
import os
import shutil

os.chdir('/content')

# Remove old code if exists
if os.path.exists('meep'):
    shutil.rmtree('meep')
    print("🧹 Cleaned up old code")

!git clone https://github.com/tyriqmiles0529-pixel/meep.git
os.chdir('meep')

print("\n📍 Code version:")
!git log -1 --oneline

# Add to Python path so we can import train_auto
import sys
sys.path.insert(0, '/content/meep')

print("\n✅ Setup complete!")
print("\nNext: Run 'Upload Models & Data' cell")

---

## 📤 Upload Models & Data

In [ ]:
# ============================================================
# API KEYS - Required for betting odds
# ============================================================

import os

# Set your API keys here (or leave empty to use environment variables)
API_SPORTS_KEY = ""  # From api-sports.io (for NBA games)
RAPIDAPI_KEY = "9ef7289093msh76adf5ee5bedb5fp15e0d6jsnc2a0d0ed9abe"  # From rapidapi.com (for TheRundown odds)
THEODDS_API_KEY = ""  # Optional: From the-odds-api.com

# Set environment variables
if API_SPORTS_KEY:
    os.environ["API_SPORTS_KEY"] = API_SPORTS_KEY
if RAPIDAPI_KEY:
    os.environ["RAPIDAPI_KEY"] = RAPIDAPI_KEY
if THEODDS_API_KEY:
    os.environ["THEODDS_API_KEY"] = THEODDS_API_KEY

print("="*70)
print("🔑 API KEY STATUS")
print("="*70)
print(f"API-Sports: {'✅ Set' if os.getenv('API_SPORTS_KEY') else '❌ Missing'}")
print(f"RapidAPI:   {'✅ Set' if os.getenv('RAPIDAPI_KEY') else '❌ Missing'}")
print(f"The Odds:   {'✅ Set' if os.getenv('THEODDS_API_KEY') else '⚠️  Optional (not set)'}")

if not os.getenv('API_SPORTS_KEY'):
    print("\n⚠️  WARNING: API_SPORTS_KEY not set - cannot fetch games")
    print("   Get key from: https://dashboard.api-sports.io/")

if not os.getenv('RAPIDAPI_KEY'):
    print("\n⚠️  WARNING: RAPIDAPI_KEY not set - cannot fetch betting odds")
    print("   Get key from: https://rapidapi.com/")
    
print("\n✅ Configuration complete")

---

## 🔑 Configure API Keys

In [ ]:
# ============================================================
# GENERATE PREDICTIONS - Uses same feature engineering as training
# ============================================================

import sys
sys.path.insert(0, '/content/meep')

import pickle
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path

print("="*70)
print("🎯 NBA PREDICTION SYSTEM")
print("="*70)

# Load models
print("\n📊 Loading models...")
models = {}
props = ['minutes', 'points', 'rebounds', 'assists', 'threes']

for prop in props:
    model_path = Path(f'models/{prop}_model.pkl')
    if model_path.exists():
        with open(model_path, 'rb') as f:
            models[prop] = pickle.load(f)
        n_features = len(models[prop].feature_names)
        print(f"   ✅ {prop.capitalize()}: {n_features} features")
    else:
        print(f"   ❌ {prop.capitalize()}: Not found")

if not models:
    raise Exception("No models loaded! Make sure you uploaded nba_models_trained.zip")

print(f"\n✅ Loaded {len(models)} models")

# Load player data and build features using EXACT SAME PIPELINE as training
print("\n🔧 Building features (using train_auto.py pipeline)...")
from train_auto import build_players_from_playerstats

# This uses the EXACT SAME feature engineering as training
players_df = build_players_from_playerstats(
    player_stats_csv='/content/PlayerStatistics.csv',
    priors_dir='/content/priors_data',
    min_season=2024,  # Only recent data for predictions
    verbose=True
)

if players_df is None or len(players_df) == 0:
    raise Exception("Failed to build features from PlayerStatistics.csv")

print(f"\n✅ Built features for {len(players_df):,} player-games")
print(f"   Features: {len(players_df.columns)} columns")

# Get most recent games for each player
print("\n🔍 Finding most recent game for each player...")
players_df = players_df.sort_values('game_date')
latest_games = players_df.groupby('player_name').tail(1)

print(f"   Found {len(latest_games)} players with recent data")

# Generate predictions
print("\n🎯 Generating predictions...")
predictions = []

for prop_name, model in models.items():
    print(f"\n   Predicting {prop_name}...")
    
    # Get features model expects
    feature_cols = model.feature_names
    
    # Check if all features exist
    missing = [f for f in feature_cols if f not in latest_games.columns]
    if missing:
        print(f"      ⚠️  Missing features: {missing[:5]}...")
        continue
    
    X = latest_games[feature_cols]
    
    # Make predictions
    preds = model.predict(X)
    
    # Store results
    for idx, (player_idx, row) in enumerate(latest_games.iterrows()):
        predictions.append({
            'player': row['player_name'],
            'team': row.get('team', 'UNK'),
            'prop': prop_name,
            'prediction': round(preds[idx], 1),
            'last_game': row['game_date']
        })

# Convert to DataFrame
pred_df = pd.DataFrame(predictions)

# Pivot to wide format (one row per player)
pred_wide = pred_df.pivot_table(
    index=['player', 'team', 'last_game'],
    columns='prop',
    values='prediction'
).reset_index()

# Sort by points prediction
pred_wide = pred_wide.sort_values('points', ascending=False)

print("\n" + "="*70)
print("📋 TOP 20 PREDICTIONS")
print("="*70)
print(pred_wide.head(20).to_string(index=False))

print("\n" + "="*70)
print("✅ PREDICTIONS COMPLETE")
print("="*70)
print(f"\nTotal players: {len(pred_wide)}")
print(f"Props predicted: {list(models.keys())}")
print("\nFeatures used: EXACT SAME as training (from train_auto.py)")
print("\nNext: Compare to betting lines to find value bets")

# ============================================================
# RUN FULL ANALYSIS - Uses riq_analyzer.py
# ============================================================

import sys
sys.path.insert(0, '/content/meep')

# Import and run riq_analyzer main function
print("="*72)
print("🎯 RUNNING FULL NBA PREDICTION ANALYSIS")
print("="*72)
print("\nThis uses riq_analyzer.py to:")
print("  1. Fetch upcoming NBA games (API-Sports)")
print("  2. Fetch betting odds (TheRundown/RapidAPI)")
print("  3. Load trained models")
print("  4. Generate predictions")
print("  5. Compare predictions vs lines")
print("  6. Calculate EV, Kelly, ELG scores")
print("  7. Build optimal parlays")
print("\n" + "="*72 + "\n")

try:
    # Import riq_analyzer and run main
    import riq_analyzer
    
    # Run main analysis
    riq_analyzer.main()
    
except ImportError as e:
    print(f"❌ Failed to import riq_analyzer: {e}")
    print("\n⚠️  Make sure you ran the Setup cell to clone the repo")
    
except Exception as e:
    print(f"❌ Analysis failed: {e}")
    import traceback
    traceback.print_exc()
    print("\n💡 Possible issues:")
    print("  - API keys not set (run API Keys cell)")
    print("  - No games today")
    print("  - API rate limits exceeded")
    print("  - Models not uploaded")

In [ ]:
# ============================================================
# UPLOAD - Models, priors data, and player statistics
# ============================================================

from google.colab import files
import os
import zipfile

os.chdir('/content')

print("="*70)
print("📤 UPLOAD ALL FILES")
print("="*70)
print("\nUpload ALL 3 files at once:")
print("  1. nba_models_trained.zip (your trained models)")
print("  2. priors_data.zip (Basketball Reference priors)")
print("  3. PlayerStatistics.csv.zip (historical data)")
print("\nUploading...\n")

uploaded = files.upload()

# Extract models
if os.path.exists('nba_models_trained.zip'):
    print("\n📦 Extracting models...")
    !unzip -q nba_models_trained.zip
    !rm nba_models_trained.zip
    print("✅ Models extracted to ./models/ and ./model_cache/")
else:
    print("⚠️  nba_models_trained.zip not found")

# Extract priors
if os.path.exists('priors_data.zip'):
    print("\n📦 Extracting priors...")
    !unzip -q priors_data.zip
    !rm priors_data.zip
    print("✅ Priors extracted to ./priors_data/")
else:
    print("⚠️  priors_data.zip not found")

# Extract player stats
if os.path.exists('PlayerStatistics.csv.zip'):
    print("\n📦 Extracting PlayerStatistics.csv...")
    !unzip -q PlayerStatistics.csv.zip
    !rm PlayerStatistics.csv.zip
    if os.path.exists('PlayerStatistics.csv'):
        size_mb = os.path.getsize('PlayerStatistics.csv') / 1024 / 1024
        print(f"✅ PlayerStatistics.csv extracted ({size_mb:.1f} MB)")
    else:
        print("❌ Extraction failed")
else:
    print("⚠️  PlayerStatistics.csv.zip not found")

# Final verification
print("\n" + "="*70)
print("🔍 FINAL VERIFICATION")
print("="*70)

if os.path.exists('models'):
    models = [f for f in os.listdir('models') if f.endswith('.pkl')]
    print(f"✅ Found {len(models)} model files")
else:
    print("❌ models/ directory not found")

if os.path.exists('priors_data'):
    priors = [f for f in os.listdir('priors_data') if f.endswith('.csv')]
    print(f"✅ Found {len(priors)} priors files")
else:
    print("❌ priors_data/ directory not found")

if os.path.exists('PlayerStatistics.csv'):
    size_mb = os.path.getsize('PlayerStatistics.csv') / 1024 / 1024
    print(f"✅ PlayerStatistics.csv ready ({size_mb:.1f} MB)")
else:
    print("❌ PlayerStatistics.csv not found")

print("\n✅ Upload complete!")
print("\nNext: Run prediction cell below")

In [ ]:
# ============================================================
# BACKTEST - Test models on recent historical games
# ============================================================

import pickle
import pandas as pd
import numpy as np
from pathlib import Path

print("="*70)
print("🧪 BACKTEST MODE")
print("="*70)

# Load trained models
print("\n📊 Loading models...")
models = {}
props = ['minutes', 'points', 'rebounds', 'assists', 'threes']

for prop in props:
    model_path = Path(f'models/{prop}_model.pkl')
    if model_path.exists():
        with open(model_path, 'rb') as f:
            models[prop] = pickle.load(f)
        print(f"   ✅ {prop.capitalize()}: {len(models[prop].feature_names)} features")
    else:
        print(f"   ❌ {prop.capitalize()}: Not found")

print(f"\n✅ Loaded {len(models)} models")

# Check model structure
print("\n🔍 Model Structure (Points):")
points_model = models.get('points')
if points_model:
    print(f"   TabNet: {'✅' if hasattr(points_model, 'tabnet') else '❌'}")
    print(f"   LightGBM: {'✅' if hasattr(points_model, 'lgbm') else '❌'}")
    print(f"   Embedding scaler: {'✅' if hasattr(points_model, 'embedding_scaler') else '❌'}")
    print(f"   Total features: {len(points_model.feature_names)}")
    
    # Show feature importance
    if hasattr(points_model, 'lgbm') and hasattr(points_model.lgbm, 'feature_importances_'):
        importances = sorted(
            zip(points_model.feature_names + [f'tabnet_emb_{i}' for i in range(24)], 
                points_model.lgbm.feature_importances_),
            key=lambda x: x[1],
            reverse=True
        )[:10]
        
        print("\n🧠 Top 10 Most Important Features:")
        for feat, imp in importances:
            emoji = "🧠" if 'tabnet_emb' in feat else "📊"
            print(f"   {emoji} {feat}: {imp:.1%}")

print("\n" + "="*70)
print("📌 TODO: Add backtest logic")
print("="*70)
print("\nNext steps:")
print("1. Load validation data from training")
print("2. Generate predictions")
print("3. Compare to actual results")
print("4. Calculate RMSE, MAE, directional accuracy")

In [ ]:
# ============================================================
# LIVE PREDICTIONS - Fetch today's games and predict
# ============================================================

from datetime import datetime
from nba_api.stats.endpoints import ScoreboardV2
import pickle
from pathlib import Path

print("="*70)
print("🎯 LIVE PREDICTION MODE")
print("="*70)

# Get today's date
today = datetime.now().strftime('%Y-%m-%d')
print(f"\n📅 Date: {today}")

# Fetch today's games
print("\n🏀 Fetching today's NBA games...")
try:
    scoreboard = ScoreboardV2(game_date=today)
    games = scoreboard.get_data_frames()[0]
    
    if len(games) == 0:
        print("   ⚠️  No games scheduled for today")
    else:
        print(f"   ✅ Found {len(games)} games")
        print("\n📋 Games:")
        for idx, game in games.iterrows():
            home = game.get('HOME_TEAM_ABBREVIATION', 'HOME')
            away = game.get('VISITOR_TEAM_ABBREVIATION', 'AWAY')
            print(f"      {away} @ {home}")
except Exception as e:
    print(f"   ❌ Error fetching games: {e}")
    games = None

# Load models
print("\n📊 Loading models...")
models = {}
props = ['minutes', 'points', 'rebounds', 'assists', 'threes']

for prop in props:
    model_path = Path(f'models/{prop}_model.pkl')
    if model_path.exists():
        with open(model_path, 'rb') as f:
            models[prop] = pickle.load(f)
        print(f"   ✅ {prop.capitalize()}")

print(f"\n✅ Loaded {len(models)} models")

print("\n" + "="*70)
print("📌 TODO: Add live prediction logic")
print("="*70)
print("\nNext steps:")
print("1. For each game, get probable starters")
print("2. Fetch recent game logs (last 10 games)")
print("3. Use train_auto.py feature engineering")
print("4. Generate predictions")
print("5. Fetch betting lines")
print("6. Find value bets (model vs line difference > threshold)")

In [ ]:
# ============================================================
# FEATURE ENGINEERING - Reuse training code
# ============================================================

# Import feature engineering from train_auto.py
# This ensures features match training exactly!

import sys
sys.path.insert(0, '/content/meep')

try:
    from train_auto import build_players_from_playerstats
    print("✅ Imported feature engineering from train_auto.py")
    print("   This ensures features match training schema exactly!")
except ImportError as e:
    print(f"❌ Failed to import: {e}")
    print("   Make sure you ran the Setup cell first")

# TODO: Add helper functions for:
# - Fetching player recent games
# - Converting API data to training schema
# - Generating predictions
# - Comparing to betting lines

In [ ]:
# ============================================================
# EVALUATION - Track accuracy and ROI
# ============================================================

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

def evaluate_predictions(y_true, y_pred, prop_name):
    """
    Calculate prediction accuracy metrics.
    
    Args:
        y_true: Actual values
        y_pred: Predicted values
        prop_name: Property being predicted (points, rebounds, etc.)
    
    Returns:
        Dict of metrics
    """
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    
    # Directional accuracy (for betting)
    # TODO: Compare to betting lines
    
    print(f"\n📊 {prop_name.upper()} Performance:")
    print(f"   RMSE: {rmse:.3f}")
    print(f"   MAE: {mae:.3f}")
    
    return {
        'rmse': rmse,
        'mae': mae,
        'prop': prop_name
    }

# TODO: Add betting performance tracking
# - Win rate (over/under)
# - ROI calculation
# - Kelly criterion sizing
# - Bankroll tracking

In [ ]:
# ============================================================
# BETTING ANALYSIS - Find value bets
# ============================================================

def find_value_bets(predictions, lines, threshold=2.0):
    """
    Find value bets where model disagrees with betting line.
    
    Args:
        predictions: Dict of player predictions {player_name: {prop: prediction}}
        lines: Dict of betting lines {player_name: {prop: line}}
        threshold: Minimum difference to consider (default: 2.0)
    
    Returns:
        List of value bets sorted by edge
    """
    value_bets = []
    
    for player, preds in predictions.items():
        if player not in lines:
            continue
        
        player_lines = lines[player]
        
        for prop, pred_value in preds.items():
            if prop not in player_lines:
                continue
            
            line = player_lines[prop]
            edge = abs(pred_value - line)
            
            if edge >= threshold:
                direction = 'OVER' if pred_value > line else 'UNDER'
                
                value_bets.append({
                    'player': player,
                    'prop': prop,
                    'prediction': pred_value,
                    'line': line,
                    'edge': edge,
                    'direction': direction,
                    'confidence': edge / threshold  # Simple confidence metric
                })
    
    # Sort by edge (highest first)
    value_bets.sort(key=lambda x: x['edge'], reverse=True)
    
    return value_bets

# TODO: Add Kelly criterion bet sizing
# TODO: Add odds fetching (API integration)
# TODO: Add historical bet tracking